In [1]:
import math
import numpy as np
import sys
import cv2
import transforms3d  # pip3 install transforms3d==0.4.1
import math
import numpy
import numpy as np
from scipy.spatial.transform import Rotation as R

In [2]:

# https://learnopencv.com/rotation-matrix-to-euler-angles/
# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R):
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype=R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6


# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R):

    assert isRotationMatrix(R)

    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])

    singular = sy < 1e-6

    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])

In [3]:
name = "500"
intrinsic_yaml_file = f"/home/ros/src/camera_utility/intrinsic/{name}.yaml"
extrinsic_yaml_file = f"/home/ros/src/camera_utility/extrinsic/{name}.yaml"

In [4]:
camera_name = name
intrinsic_yaml = cv2.FileStorage(intrinsic_yaml_file, cv2.FILE_STORAGE_READ)
extrinsic_yaml = cv2.FileStorage(extrinsic_yaml_file, cv2.FILE_STORAGE_READ)

# https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html
# https://www.geeksforgeeks.org/calibratecamera-opencv-in-python/

# 9 items: Camera matrix
in_K = intrinsic_yaml.getNode("K").mat()
# 5 items: Distortion coefficients
in_D = intrinsic_yaml.getNode("D").mat()

in_xi = intrinsic_yaml.getNode("xi").mat()  # 0

# 3 items
in_image_shape = intrinsic_yaml.getNode("image_shape").mat()

# 9 items: rotation vectors
ex_rot_mat = extrinsic_yaml.getNode("rot_mat").mat()

# 3 items:  translation vectors
ex_t_vec = extrinsic_yaml.getNode("t_vec").mat()

# 12 items:
ex_camera_matrix_p = extrinsic_yaml.getNode("camera_matrix_p").mat()

# 3 items: xyz position
ex_position = extrinsic_yaml.getNode("position").mat()


In [5]:

# http://sdformat.org/tutorials?tut=specify_pose
# The elements x y z specify the position vector (in meters), and the elements roll pitch yaw are Euler angles (in radians) that specify the orientation, which can be computed by an extrinsic X-Y-Z rotation

x_ = ex_position[0][0]
y_ = ex_position[1][0]
z_ = ex_position[2][0]

invR = numpy.linalg.inv(ex_rot_mat)
neg_invR = numpy.negative(invR)
neg_invR_t = numpy.dot(neg_invR, ex_t_vec)
x = neg_invR_t[0][0]
y = neg_invR_t[1][0]
z = neg_invR_t[2][0]

# other possible _AXES2TUPLE : https://github.com/matthew-brett/transforms3d/blob/main/transforms3d/euler.py#L148
# https://www.andre-gaschler.com/rotationconverter/

height = int(in_image_shape[0][0])
width = int(in_image_shape[1][0])

# 84 degree to 1.46607657 radians
# http://sdformat.org/spec?ver=1.11&elem=sensor#camera_distortion

k1 = in_D[0][0]
k2 = in_D[1][0]
p1 = in_D[2][0]
p2 = in_D[3][0]
k3 = in_D[4][0]

# [OpenCV](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html) uses five parameters, known as distortion coefficients given by like this: `k1, k2, p1, p2 , k3 # pay attention to the order`

px = in_K[0][2]
py = in_K[1][2]
fx = in_K[0][0]
fy = in_K[1][1]
fov_x = 2 * math.atan(width / (2 * fx))
fov_y = 2 * math.atan(height / (2 * fy))
# The FoV is
# ~84 degrees horizontal
# ~53 degrees vertical
# print(fov_x, fov_y) # 53.550787735526455 83.92250895589785

fovhor = fov_x
aspect_ratio = width / height

# rpy, R, Q, Qx, Qy, Qz = cv2.RQDecomp3x3(ex_rot_mat)

# R_temp = np.array([
#     [0, -1 ,0],
#     [0, 0, -1],
#     [1, 0, 0]
# ])
R_world_to_cam = ex_rot_mat
R_cam_to_gazebocam = np.array(
    [[0.0, 0.0, 1.0], [-1.0, 0.0, 0.0], [0.0, -1.0, 0.0]]
)
R_world_to_gazebocam = np.matmul(R_cam_to_gazebocam, R_world_to_cam)
R_gazebocam_to_world = np.linalg.inv(R_world_to_gazebocam)

# First, it turns out that the cameras in gazebo use another coordinate system than what is usually used for cameras (e.g., assumed by the calibration methods in opencv).
# Therefore, a rotation matrix R_cam_to_gazebocam to account is introduced.
# Second, rather than computing the Euler angles from R_world_to_gazebocam, (which was our initial attempt), the Euler angles must be computed from R_gazebocam_to_world.

r, p, w = rotationMatrixToEulerAngles(R_gazebocam_to_world)


We have the data now we want to get the resulting rpy. Test it. And update it. THen we convert our RPY into rotation and inverse the calculations to get our rot matrix.


In [6]:
# Existing values
r
p
w
ex_rot_mat

array([[ 2.22044605e-16, -1.00000000e+00, -2.22044605e-16],
       [ 0.00000000e+00,  2.22044605e-16, -1.00000000e+00],
       [ 1.00000000e+00,  2.22044605e-16,  0.00000000e+00]])

In [7]:
def rot_to_rpy(rot_matrix_input):
    R_world_to_cam = rot_matrix_input
    R_cam_to_gazebocam = np.array(
        [[0.0, 0.0, 1.0], [-1.0, 0.0, 0.0], [0.0, -1.0, 0.0]]
    )
    R_world_to_gazebocam = np.matmul(R_cam_to_gazebocam, R_world_to_cam)
    R_gazebocam_to_world = np.linalg.inv(R_world_to_gazebocam)

    r, p, w = rotationMatrixToEulerAngles(R_gazebocam_to_world)
    return r,p,w

In [8]:
#rr,pp,ww = 0 ,3.14, 0.786 # 0, PI 180, PI/2 45
rr,pp,ww = 0, 1.571, 0  # 0, 0, PI/2

rotation_matrix_new = R.from_euler('xyz', [rr, pp, ww]).as_matrix()
rr,pp,ww = rot_to_rpy(rotation_matrix_new)
print(rr,pp,ww)

-1.5707963267948966 0.00020367320510334213 -3.141592653589793


In [9]:
print(r,p,w)

2.220446049250313e-16 -4.930380657631324e-32 2.220446049250313e-16


In [10]:


#rotation_matrix_new = rotation_matrix_new.T @ R_world_to_gazebocam
print(rotation_matrix_new)
#print(np.allclose(rotation_matrix_new, ex_rot_mat))
# I want rotation_matrix_new == ex_rot_mat 
res = rot_to_rpy(rotation_matrix_new)

[[-2.03673204e-04  0.00000000e+00  9.99999979e-01]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-9.99999979e-01  0.00000000e+00 -2.03673204e-04]]


In [11]:
res

(-1.5707963267948966, 0.00020367320510334213, -3.141592653589793)

In [12]:
print(rotation_matrix_new.T @ R_world_to_gazebocam )
print("--------")
print(ex_rot_mat)

diff = ex_rot_mat @ rotation_matrix_new.T
print("Should be identity:\n", diff)
print("Close?", np.allclose(diff, np.eye(3), atol=1e-6))

[[-2.03673204e-04  2.21999376e-16 -9.99999979e-01]
 [-2.22044605e-16  1.00000000e+00  2.22044605e-16]
 [ 9.99999979e-01  2.22089825e-16 -2.03673204e-04]]
--------
[[ 2.22044605e-16 -1.00000000e+00 -2.22044605e-16]
 [ 0.00000000e+00  2.22044605e-16 -1.00000000e+00]
 [ 1.00000000e+00  2.22044605e-16  0.00000000e+00]]
Should be identity:
 [[-2.22089825e-16 -1.00000000e+00 -2.21999376e-16]
 [-9.99999979e-01  2.22044605e-16  2.03673204e-04]
 [-2.03673204e-04  2.22044605e-16 -9.99999979e-01]]
Close? False


Get the t_vec given that i have x,y,z and rotation_matrix

In [13]:
C = np.array([[3.5], [0], [2.0]])   # camera center in world coords
t_vec = -rotation_matrix_new @ C
t_vec

array([[-1.9992871 ],
       [ 0.        ],
       [ 3.50040727]])

Get everything i need:

In [14]:
def generate_extrinsic_data(camera_id, rpy=[0,0,0], world_pose=[[3.5], [0], [2.0]]):
    rpy_ordered = [rpy[1],rpy[2],-rpy[0]] #[rpy[2],rpy[0],rpy[1]]
    rotation_matrix_new = R.from_euler('xyz', rpy_ordered).as_matrix()
    #rotation_matrix_new = rotation_matrix_new @ rot_convert_matrix
    print("---------")
    print(f"cam: {camera_id}" )
    print("Rotation matrix:", *rotation_matrix_new.flatten(), sep=", ")

    C = np.array(world_pose)   # camera center in world coords
    t_vec = -rotation_matrix_new @ C 
    print(f"t_vec: [{t_vec[0][0]}, {t_vec[1][0]}, {t_vec[2][0]}]")
    print("---------")

x = z, rotates roll \
y = x, rotates pitch \
z = y, rotates yaw

In [15]:
# Old output
generate_extrinsic_data("500", [0, math.pi/2, -math.pi/2], [[3.5], [0], [2.0]]) # base
generate_extrinsic_data("501", [0, math.pi/2, 0],          [[5.5], [-2.0], [2.0]])  # to the right, facing left 90°
generate_extrinsic_data("502", [0, math.pi/2, math.pi/2],  [[7.5], [0], [2.0]]) # To the left facing abit right ## note: set Y but changed Z counterclockwise
generate_extrinsic_data("503", [0, math.pi/2, math.pi],    [[5.5], [2.0], [2.0]]) # move to left left ## note: set Z but changed X clockwise

---------
cam: 500
Rotation matrix:, 2.220446049250313e-16, -1.0, -2.220446049250313e-16, 0.0, 2.220446049250313e-16, -1.0, 1.0, 2.220446049250313e-16, 0.0
t_vec: [-3.3306690738754696e-16, 2.0, -3.5]
---------
---------
cam: 501
Rotation matrix:, 1.0, 0.0, 0.0, 0.0, 2.220446049250313e-16, -1.0, 0.0, 1.0, 2.220446049250313e-16
t_vec: [-5.5, 2.0000000000000004, 1.9999999999999996]
---------
---------
cam: 502
Rotation matrix:, 2.220446049250313e-16, 1.0, 2.220446049250313e-16, 0.0, 2.220446049250313e-16, -1.0, -1.0, 2.220446049250313e-16, 0.0
t_vec: [-2.1094237467877974e-15, 2.0, 7.5]
---------
---------
cam: 503
Rotation matrix:, -1.0, 1.2246467991473532e-16, 2.465190328815662e-32, 0.0, 2.220446049250313e-16, -1.0, -1.2246467991473532e-16, -1.0, -2.220446049250313e-16
t_vec: [5.5, 1.9999999999999996, 2.0000000000000013]
---------
